In [ ]:
import numpy as np
import random

class AntColonyOptimization:
    def __init__(self, distances, num_ants, num_iterations, alpha, beta, evaporation_rate, Q):
        self.distances = distances
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.Q = Q
        self.num_cities = len(distances)
        self.pheromones = np.ones((self.num_cities, self.num_cities))
        self.best_path = None
        self.best_distance = float('inf')

    def run(self):
        for _ in range(self.num_iterations):
            paths = self.construct_paths()
            self.update_pheromones(paths)
            self.update_best_path(paths)

    def construct_paths(self):
        paths = []
        for _ in range(self.num_ants):
            visited = [False] * self.num_cities
            current_city = random.randint(0, self.num_cities - 1)
            visited[current_city] = True
            path = [current_city]

            for _ in range(self.num_cities - 1):
                next_city = self.select_next_city(current_city, visited)
                path.append(next_city)
                visited[next_city] = True
                current_city = next_city

            path.append(path[0])
            paths.append(path)

        return paths

    def select_next_city(self, current_city, visited):
      pheromone = self.pheromones[current_city]
      distance = 1.0 / (self.distances[current_city] + 1e-8)  # Add a small constant
      probabilities = (pheromone ** self.alpha) * (distance ** self.beta)
      probabilities[visited] = 0
      probabilities /= probabilities.sum()
      next_city = np.random.choice(range(self.num_cities), p=probabilities)
      return next_city

    def update_pheromones(self, paths):
        self.pheromones *= (1 - self.evaporation_rate)
        for path in paths:
            distance = self.calculate_distance(path)
            for i in range(self.num_cities):
                j = (i + 1) % self.num_cities
                self.pheromones[path[i], path[j]] += self.Q / distance

    def update_best_path(self, paths):
        for path in paths:
            distance = self.calculate_distance(path)
            if distance < self.best_distance:
                self.best_path = path
                self.best_distance = distance

    def calculate_distance(self, path):
        distance = 0
        for i in range(self.num_cities):
            j = (i + 1) % self.num_cities
            distance += self.distances[path[i], path[j]]
        return distance

In [ ]:
# Example usage
# distances = np.array([
#     [0, 2, 9, 10],
#     [1, 0, 6, 4],
#     [15, 7, 0, 8],
#     [6, 3, 12, 0]
# ])
distances = np.array([
    [0, 10, 15, 20, 5],
    [10, 0, 35, 25, 15],
    [15, 35, 0, 30, 20],
    [20, 25, 30, 0, 10],
    [5, 15, 20, 10, 0]
])

In [ ]:
aco = AntColonyOptimization(distances, num_ants=10, num_iterations=100, alpha=1, beta=2, evaporation_rate=0.5, Q=100)
aco.run()

In [ ]:
print("Best path:", aco.best_path)
print("Best distance:", aco.best_distance)

Best path: [4, 2, 0, 1, 3, 4]
Best distance: 80
